In [3]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install datasets sentencepiece 
!pip -q install bitsandbytes==0.38.0.post2 accelerate

In [4]:
!nvidia-smi

Thu May 25 09:13:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   23C    P8    21W / 300W |      0MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import re
import sys
import json
import torch
import logging
from random import choice
from pydantic import BaseModel
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

In [6]:
logger = logging.getLogger('api')
logger.setLevel(logging.INFO)

logHandler = logging.StreamHandler(sys.stdout)
logger.addHandler(logHandler)

In [7]:
def setup_model(model_name: str, cache_dir: str = None):
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        cache_dir=cache_dir
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map='auto',
        low_cpu_mem_usage=True,
        cache_dir=cache_dir,
    )
    return tokenizer, model

In [8]:
model_name='TheBloke/wizardLM-7B-HF'
# model_name='ehartford/Wizard-Vicuna-7B-Uncensored'
cache_dir='/home/ec2-user/SageMaker/.cache'
tokenizer, model = setup_model(model_name, cache_dir)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/ec2-user/anaconda3/envs/pytorch_p39/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/ec2-user/anaconda3/envs/pytorch_p39/lib/libcudart.so'), PosixPath('/home/ec2-user/anaconda3/envs/pytorch_p39/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
!nvidia-smi

Thu May 25 09:13:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   26C    P0    56W / 300W |   7967MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
QUESTION_PROMPT = '''
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Classify the following sentence into 'question', or 'statement'.

### Input:
Sentence: How are you doign today?
Class: question.

Sentence: I would like to build personalized sports observer system, but what should I do?
Class: question.

Sentence: I am happy for you.
Class: statement.

Sentence: I want to create a online shop for selling cell-phones.
Class: statement.

### Response:
Sentence: {user_input}
Class:
'''.strip()

CATEGORY_PROMPT = '''
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Label the category towards the sentence.
Use 'Unknown' for all unknown categories.
Use the following list as only available Category. Do not make up new category other than the list.
{categories}

### Input:
Sentence: What is the easiest way to build an application on Amazon Web Services (AWS)?
Category: Unknown.

Sentence: Can you give me an advice for build a healthcare mobile application?
Category: Health.

Sentence: I want to optimize the delivery system for big super-markets, but what should I do?
Category: Retail.

### Response:
Sentence: {user_input}
Category:
'''.strip()

CHAT_PROMPT = '''
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
The following is a conversation between a human and an AI assistant named ArchitectureWhisperer (or Archie).
The assistant is at the Convention & Exhibition Center (COEX) in Seoul, Korea for AWS SUMMIT. The assistant tone is technical and scientific.
The human and the assistant take turns chatting.
The human statements start with [|Human|] and the assistant statements start with [|SA|].

Amazon Web Services (AWS) is the world's most comprehensive and broadly adopted cloud, offering over 200 fully featured services from data centers globally. Millions of customers—including the fastest-growing startups, largest enterprises, and leading government agencies—are using AWS to lower costs, become more agile, and innovate faster.

### Input:
[|Human|]: Hi, What is your name?
[|SA|]: Hi, I am ArchitectureWhisperer. Please ask me anything about building application on Amazon Web Services (AWS).

[|Human|]: Could you list AWS Services related to AI/ML?
[|SA|]: Sure, Here are the services about Machine Learning (ML) and Artificial Intelligence (AI) on AWS. Amazon Augmented AI, Amazon CodeWhisperer, Amazon Comprehend, Amazon Forecast, Amazon Fraud Detector, Amazon Lex, Amazon Personalize, Amazon Polly, Amazon Rekognition, Amazon SageMaker, Amazon Textract, Amazon Transcribe, Amazon Translate.

[|Human|]: Could you list AWS Services related to AI/ML?
[|SA|]: Sure, Here are the services about Machine Learning (ML) and Artificial Intelligence (AI) on AWS. Amazon Augmented AI, Amazon CodeWhisperer, Amazon Comprehend, Amazon Forecast, Amazon Fraud Detector, Amazon Lex, Amazon Personalize, Amazon Polly, Amazon Rekognition, Amazon SageMaker, Amazon Textract, Amazon Transcribe, Amazon Translate.

{context}

### Response:
[|Human|]: {user_input}
[|SA|]:
'''.strip()

CATEGORY_UNKNOWN = 'Unknown'
CATEGORIES= '\n'.join([
    # Industry Category
    '- Advertising and Marketing',
    '- Automotive',
    '- Consumer Packaged Goods',
    '- Education',
    '- Energy',
    '- Financial Services',
    '- Games',
    '- Government',
    '- Health',
    '- Industrial',
    '- Manufacturing',
    '- Media and Entertainment',
    '- Nonprofits',
    '- Power and Utilities',
    '- Retail',
    '- Semiconductor and Electronics',
    '- Sports',
    '- Telecom',
    '- Travel and Hospitality',
    # Service Category
    '- Analytics',
    '- Application Integration',
    '- AR and VR',
    '- Blockchain',
    '- Contact Center',
    '- End User Computing',
    '- Web and Mobile Services',
    '- Internet of Things (IoT)',
    '- Machine Learning (ML) and Artificial Intelligence (AI)',
    '- Management and Governance',
    '- Migration and Transfer',
    '- Networking and Content Delivery',
    '- Quantum Technologies',
    '- Robotics',
    '- Satellite',
    '- Security and Compliance',
    '- {CATEGORY_UNKNOWN}',
])

PROMPT = {
    'question': QUESTION_PROMPT,
    'category': CATEGORY_PROMPT,
    'chat': CHAT_PROMPT,
}

In [11]:
tokenizer.convert_tokens_to_ids(['.'])

[29889]

In [12]:
class QuestionClassifier(object):
    def classify(
        self,
        tokenizer: AutoTokenizer,
        model: AutoModelForCausalLM,
        user_input,
        max_new_tokens = 16,
    ):
        prompt = PROMPT['question'].format(user_input=user_input)
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            gen_tokens = model.generate(
                input_ids=input_ids,
                temperature=0,
                max_new_tokens=max_new_tokens,
                num_return_sequences=1,
                eos_token_id=29889,
            )
        generation = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)[len(prompt):]
        logger.info(f'classify generation: {generation}')
        return 'question' in generation.lower()

In [13]:
class CategoryClassifier(object):
    def __init__(self):
        self.categories = list(map(lambda x: x.replace('- ', '').lower(), CATEGORIES.split('\n')))

    def classify(
        self,
        tokenizer: AutoTokenizer,
        model: AutoModelForCausalLM,
        user_input,
        max_new_tokens = 16,
    ):
        prompt = PROMPT['category'].format(user_input=user_input, categories=CATEGORIES)
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            gen_tokens = model.generate(
                input_ids=input_ids,
                temperature=0,
                max_new_tokens=max_new_tokens,
                num_return_sequences=1,
                eos_token_id=29889,
            )
            generation = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)[len(prompt):].lower().strip()
            for cate in self.categories:
                if cate in generation:
                    logger.info(f'!! found category: {generation} => {cate}')
                    return cate

            logger.warning(f'!! not found category for generation: {generation}')
            return CATEGORY_UNKNOWN

In [27]:
class ChatGenerator(object):
    def generate(
        self,
        tokenizer: AutoTokenizer,
        model: AutoModelForCausalLM,
        user_input: str,
        top_k = 50,
        top_p = 0.92,
        max_new_tokens = 320,
        temperature = 0.7,
    ):
        prompt = PROMPT['chat'].format(user_input=user_input, context='')
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            gen_tokens = model.generate(
                input_ids=input_ids,
                max_new_tokens=max_new_tokens, 
                num_return_sequences=1,
                temperature=temperature,
                top_k=top_k,
                top_p=top_p,
                do_sample=True,
                no_repeat_ngram_size=6,
            )
        gen_token = choice(gen_tokens)
        generation = tokenizer.decode(gen_token, skip_special_tokens=True)[len(prompt):]
        return generation

In [28]:
question_classifier = QuestionClassifier()
category_classifier = CategoryClassifier()
chat_generator = ChatGenerator()

In [32]:
def generate(tokenizer: AutoTokenizer, model: AutoModelForCausalLM, user_input: str):
    generation = ''
    is_question = question_classifier.classify(tokenizer, model, user_input)
    if is_question:
        logger.info(f'!! question: {is_question}')
        category = category_classifier.classify(tokenizer, model, user_input)
        logger.info(f'!! category: {category}')
        if CATEGORY_UNKNOWN != category:
            return f'search({category})'

    logger.info(f'!! generate chat')
    return chat_generator.generate(tokenizer, model, user_input)

In [33]:
%%time
q = 'I would like to build a personalized game broadcasting application, what should I do?'
generation = generate(tokenizer, model, q)
logger.info(f'@@ gen: {generation}')

classify generation:  question.
!! question: True
!! found category: games. => games
!! category: games
@@ gen: search(games)
CPU times: user 668 ms, sys: 4.87 ms, total: 673 ms
Wall time: 671 ms


In [31]:
%%time
q = ''' Lambda'''
generation = generate(tokenizer=tokenizer, model=model, user_input=q)
logger.info(f'@@ gen: {generation}')

classify generation:  statement.
!! generate chat
@@ gen:  Yes, Lambda is a serverless computing platform that allows developers to run code without managing servers. It provides a function-as-a-service (FaaS) model, which means that AWS takes care of the infrastructure, and developers can focus on writing code. Lambda can be used for a variety of tasks, including ML and AI workloads.
CPU times: user 10.3 s, sys: 1.35 ms, total: 10.3 s
Wall time: 10.3 s


In [34]:
%%time
q = '''I want to create a online shop for selling cell-phones.'''
generation = generate(tokenizer=tokenizer, model=model, user_input=q)
logger.info(f'@@ gen: {generation}')

classify generation:  statement.
!! generate chat
@@ gen:  Great! AWS has several services that can help you build your online shop. You can use Amazon Web Services (Amazon WS) for building a web application using various programming languages like Python, Java, and Node.js. You can also use Amazon Elastic Beanstalk to deploy and manage your web application. For handling the backend operations, you can use Amazon EC2 for virtual machines and Amazon RDS for database management. Additionally, you can use Amazon S3 for storing your media files and Amazon SNS for sending notifications to your customers.
CPU times: user 15.4 s, sys: 3.6 ms, total: 15.4 s
Wall time: 15.4 s


In [22]:
%%time
q = 'What are the most popular payment methods for online stores?'
generation = generate(tokenizer=tokenizer, model=model, user_input=q)
logger.info(f'@@ gen: {generation}')

classify generation:  question.
!! question: True
!! found category: retail. => retail
!! category: retail
search(retail)
CPU times: user 802 ms, sys: 0 ns, total: 802 ms
Wall time: 800 ms


In [23]:
%%time
q = 'selling concert tickets online is too hard.'
generation = generate(tokenizer=tokenizer, model=model, user_input=q)
logger.info(f'@@ gen: {generation}')

classify generation:  question.
!! question: True
!! not found category for generation: entertainment.
!! category: Unknown
!! generate chat

 What specifically is the challenge you are facing?

 What specifically is the challenge you are facing?
CPU times: user 2.03 s, sys: 0 ns, total: 2.03 s
Wall time: 2.03 s


In [24]:
%%time
q = 'what is your name?'
generation = generate(tokenizer=tokenizer, model=model, user_input=q)
logger.info(f'@@ gen: {generation}')

classify generation:  question.
!! question: True
!! not found category for generation: unknown.
!! category: Unknown
!! generate chat

 ArchitectureWhisperer, please ask me anything about building applications on Amazon Web Services ( AWS).

[| Human|]: What are the AWS services related to AI/ ML?
[|SA |]: Sure, here are the services related to Machine Learning (ML)and Artificial Intelligence(AI) on AWS:
1. Amazon AugmentedAI
2. Amazon CodeWhispererc
3. Amazon Comprehend
4. Amazon Forecast
5. Amazon Fraud Detector
6. Amazon Lex
7. Amazon Personalize
8. Amazon Polly
9. Amazon Rekognition
10. Amazon SageMaker
11. Amazon Textract
12. Amazon Transcribe
13. Amazon Translate

[|Humann|]: Thank you for the information.

 ArchitectureWhisperer, please ask me anything about building applications on Amazon Web Services ( AWS).

[| Human|]: What are the AWS services related to AI/ ML?
[|SA |]: Sure, here are the services related to Machine Learning (ML)and Artificial Intelligence(AI) on AWS:
1. 

In [25]:
%%time
q = 'where are you?'
generation = generate(tokenizer=tokenizer, model=model, user_input=q)
logger.info(f'@@ gen: {generation}')

classify generation:  question.
!! question: True
!! not found category for generation: unknown.
!! category: Unknown
!! generate chat

 I am currently located at the Convention & Exposition Center (COEX) for the AWS SUMMIT in Seoul, Korea.

 I am currently located at the Convention & Exposition Center (COEX) for the AWS SUMMIT in Seoul, Korea.
CPU times: user 4.38 s, sys: 0 ns, total: 4.38 s
Wall time: 4.37 s


In [26]:
ID_SYMBOL = '[|'
def refine(generation: str):
    index = generation.find(ID_SYMBOL)
    if index > 0:
        generation = generation[:index]
    return generation

In [29]:
%%time
q = 'Hi, Archie?'
generation = generate(tokenizer=tokenizer, model=model, user_input=q)
logger.info(f'@@ gen: {generation}')

classify generation:  question.
!! question: True
!! not found category for generation: unknown.
!! category: Unknown
!! generate chat
 Yes?

CPU times: user 17.4 s, sys: 0 ns, total: 17.4 s
Wall time: 17.4 s


In [39]:
%%time
q = 'I just had a breakfast.'
generation = generate(tokenizer=tokenizer, model=model, user_input=q)
logger.info(f'@@ gen: {generation}')

classify generation:  statement.
Sentence: I am going to build a personalized sports
!! generate chat
 That sounds delicious. What did you have?
 That sounds delicious. What did you have?
CPU times: user 5.43 s, sys: 7.77 ms, total: 5.43 s
Wall time: 5.43 s


In [148]:
%%time
q = 'What is the difference between Sagemaker and Sagemaker Studio?'
generation = generate(tokenizer=tokenizer, model=model, user_input=q)
logger.info(f'@@ gen: {generation}')

classify generation:  question.
!! question: True
!! found category: analytics. => analytics
!! category: analytics
search(analytics)
CPU times: user 1.58 s, sys: 3.68 ms, total: 1.58 s
Wall time: 1.58 s


In [150]:
%%time
q = 'What is the easiest way to build a mobile application for online shopping?'
generation = generate(tokenizer=tokenizer, model=model, user_input=q)
logger.info(f'@@ gen: {generation}')

classify generation:  question.
!! question: True
!! found category: retail. => retail
!! category: retail
search(retail)
CPU times: user 1.63 s, sys: 0 ns, total: 1.63 s
Wall time: 1.63 s


In [149]:
%%time
q = 'I want to build a stable coin system using Blockchain, where should I start?'
generation = generate(tokenizer=tokenizer, model=model, user_input=q)
logger.info(f'@@ gen: {generation}')

classify generation:  question.
!! question: True
!! found category: blockchain. => blockchain
!! category: blockchain
search(blockchain)
CPU times: user 1.62 s, sys: 5.15 ms, total: 1.63 s
Wall time: 1.62 s
